## Classes pour l'analyse des résultats SYGIVRE
<br>

#### AUTEUR : Matisse Larocque [ LARM09069805 ]
#### VERSION : juillet 2025
##### Dans le cadre du stage STM5000, été 2025

<br>
<br>

__Notebook pour les classes SygivreStation et VerglasEvent, qui visent à pouvoir obtenir rapidement des graphs et des informations sur des stations pour des évènements de verglas.__
<br><br>


### IMPORTS

In [1]:
# IMPORTATION DE LIBRAIRIES
import numpy as np
import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import xarray
# import netCDF4
# from dask.array import average
# from shapely.geometry import Polygon
# from matplotlib.patches import Rectangle
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import math

import datetime

import pandas as pd
import os

# Script maison de Rajaonarivelo (2008)
import hq_algo

# Classe pour stations sygivres 
#### Fonctionne localement, les variables pour les paths doivent être modifiées si la classe est exportée ailleurs. 
##### TODO : Détailler les chemins relatifs nécessaires au fonctionnement de la classe
##### UTILITÉ : Obtenir une variable csv d'une station pour une année, extraire des csv pour des évènements précis

In [2]:
class SygivreStation():
    #Informations pour chemins relatifs vers les données
    # SERVEUR POUR SAUVEGARDER LES DONNÉES
    data_disk ="/chinook"

    # NOM DE MES DOSSIERS PERSO
    me = "larocque"


    # DIRECTORY VERS HOME/LAROCQUE ( mes dossiers )
    home_dir = os.path.expanduser("~")

    # DIRECTORY VERS MON DOSSIER DE DONNÉES DANS CHINOOK
    data_dir = os.path.join(data_disk, me)

    # DIRECTORY VERS MON DOSSIER DE DONNÉES HQ (par stations) DANS CHINOOK
    hq_data_dir = os.path.join(data_disk, me, "DATA_HQ","DATA_RS")

    #DIRECTORY VERS LE METADATA HQ
    hq_metadata_dir = os.path.join(data_disk, me, "METADATA_HQ")
    
#     # CONSTANTES UTILES
#     r_35mm = (35/2)*(1/1000) #rayon en [m], câble de D=35 mm
#     r_12mm = (12.5/2)*(1/1000) #rayon en [m], câble de D=12.5 mm
#     r_315kV = 0.032/2 #rayon en [m] pour ligne électrique de 315 kV (estimé pour un D de [1; 1.5] po)
#     rho_i = 900 # kg/m^3
#     pi = math.pi
#     wirel = 1 #m, longueur d'1m de fil
    
    #**************************
    
    #       FONCTIONS
    
    #**************************
    #=================
    #CONSTRUCTEUR
    #=================
    def __init__(self, station, year):
        
        stations_existantes = self.get_all_stations() 
        
        #Validation de la station (au besoin)
        while station not in stations_existantes:
            print("Station non-disponible !")
            print("Stations existantes : ", ", ".join(stations_existantes))
            station = input("Station choisie : ")
        
        annees_disponibles = self.get_available_years(station)
        
        #Validation de l'année (au besoin)
        while str(year) not in annees_disponibles :
            print("Année non-disponible !")
            print("Années disponibles pour la station :",", ".join(annees_disponibles))
            year = input("Année choisie : ")
        
        #Initialisation des variables principales de classe
        self.station = station
        self.year = str(year)
        self.file = self.set_file()
        
    #==================================
    # IDENTIFICATION PLUIE VERGLAÇANTE
    #==================================
    
    # TODO : Classe pour un objet évènement de pluie (dict d'information)
    # Fonction pour identifier début (datetime), fin (datetime), nombre d'heures
    # En fonction des colonnes ou 'NA' > 0 (début) 
    # jusqu'à 6 heures consécutives ou 'NA' = 0 (fin)
    # Le nombre d'heures est le nombre total de lignes pour un évènement
    # Getters pour début / fin / nombre d'heures
    
        
    #=================
    #SETTERS
    #=================
    #Choisir la station
    def set_station(self, station):
        self.station = station
    
    #Choisir l'année
    def set_year(self, year):
        self.year = year
    
    #Charger le fichier
    def set_file(self):
        file = self.get_station_csv()
        file['timestamp'] = pd.to_datetime(file['timestamp'])
        file.set_index('timestamp', inplace=True)
        file['NA']=file['VERGLAS'].diff() #NA : Nombre d'Alarmes durant l'heure
        file = file[['VERGLAS','NA','TEMPEXT']]
        
        return file
    
    #Choisir les dates de début et fin d'un évènement à analyser
    # *** Entrer les débuts et fins comme des tuples de int (AAAA,MM,DD) 
    def set_event_dates(self,start,end):
        #Début, Y, m, d
        start_y=start[0]
        start_m=start[1]
        start_d=start[2]
        
        #Fin, Y, m, d
        end_y=end[0]
        end_m=end[1]
        end_d=end[2]
        
        # Définir le début et la fin de l'évènement
        start = pd.to_datetime( datetime.datetime(start_y, start_m, start_d) )
        end = pd.to_datetime(datetime.datetime(end_y, end_m, end_d) )

        # Conserver seulement 'VERGLAS' dans l'intervalle de temps voulue
        self.event_file = self.file[ (self.file.index > start) & (self.file.index < end) ]
        
        return self.event_file
        
        
    
    #=================
    #GETTERS
    #=================
    # Obtenir le nom de la station
    def get_station(self):
        return self.station
    
    #Obtenir les stations au directory
    def get_all_stations(self):
        stations = os.listdir(SygivreStation.hq_data_dir)
        stations.remove(".DS_Store")
        return stations
    
    #Obtenir les années disponibles pour la station : 
    def get_available_years(self, station):
        years = os.listdir(os.path.join(SygivreStation.hq_data_dir, station))
        years.remove(".DS_Store")
        return years

    #Obtenir la station
    def get_station_csv(self):
        version = "v1_20250529"
        extention = ".csv"
        file_suffix = f"_{self.year}_{version}{extention}"
        # Assume only one version per STATION/YEAR/*.csv
        return pd.read_csv(
            os.path.join(
                SygivreStation.hq_data_dir, self.station, self.year, self.station+file_suffix ) )
        
        
    
    #Obtenir les coordonnées de la station
    def get_station_coord(self):
        # print(station)
        metadata_df = pd.read_csv(
            os.path.join( 
                SygivreStation.hq_metadata_dir,"stations_lat_lon.csv" ) )
        # print(metadata_df)
        station_line = metadata_df[metadata_df["Name"] == self.station]
        # print(station_line)
        lat = station_line["Latitude"]
        # print(lat)
        lon = station_line["Longitude"]
        # print(lon)
        #return lon, lat
        return float(lon.iloc[0]), float(lat.iloc[0])
    
    
        
    

# Classe évènements de verglas 
#### Fonctionne localement, les variables pour les paths doivent être modifiées si la classe est exportée ailleurs. 
##### TODO : Détailler les chemins relatifs nécessaires au fonctionnement de la classe
##### UTILITÉ : prendre un df d'évènement de verglas, obtenir les graphes d'accumulation selon Druez ou Savadjiev, et les heures de verglas comptabilisées


#### Version corrigée par IA de ma classe

In [3]:
class VerglasEvent():
    """
    Classe de traitement d’un fichier dévénement de givre/verglas sur des lignes électriques.
    Calcule l’évolution du rayon de câble en fonction de différentes équations issues de la littérature.
    """

    # CONSTANTES UTILES
    r_35mm = (35 / 2) * (1 / 1000)  # rayon en m, D=35 mm
    r_12mm = (12.5 / 2) * (1 / 1000)  # rayon en m, D=12.5 mm
    r_315kV = (115/2)/1000  # rayon estimé en m pour 315kV
    rho_i = 900  # kg/m^3 (verglas)
    rho_r_min = 300  # kg/m^3 (rime, min density)
    rho_r_max = 800 #kg/m^3 (rime, max density)
    
    pi = math.pi
    wirel = 1  # m, longueur de fil utilisée
    #=================
    # CONSTRUCTEUR
    #=================

    def __init__(self, event_file, station):
        self.event_file = event_file.copy()
        self.station = station

        # === VERGLAS ===
        self.event_file['DRUEZ1'] = self.event_file['NA'].apply(self.eq_1)
        self.event_file['DRUEZ2'] = self.event_file['NA'].apply(self.eq_2)
        self.event_file['SAVADJIEV1'] = self.event_file['NA'].apply(self.eq_3)

        self._accumulate_radius('R35_FzRa', 'DRUEZ1', self.r_35mm, self.rho_i)
        self._accumulate_radius('R12_FzRa', 'DRUEZ2', self.r_12mm, self.rho_i)
        self._accumulate_radius('R315_FzRa', 'SAVADJIEV1', self.r_315kV, self.rho_i)

        # === GIVRE ===
        self.event_file['DRUEZ1_RIME'] = self.event_file['NA'].apply(self.eq_4)
        self.event_file['DRUEZ2_RIME'] = self.event_file['NA'].apply(self.eq_5)
        self.event_file['SAVADJIEV1_RIME'] = self.event_file['NA'].apply(self.eq_6)

        # DENSITÉ MIN
        self._accumulate_radius('R35_min_Rime', 'DRUEZ1_RIME', self.r_35mm, self.rho_r_min)
        self._accumulate_radius('R12_min_Rime', 'DRUEZ2_RIME', self.r_12mm, self.rho_r_min)
        self._accumulate_radius('R315_min_Rime', 'SAVADJIEV1_RIME', self.r_315kV, self.rho_r_min)
        
        # DENSITÉ MAX
        self._accumulate_radius('R35_max_Rime', 'DRUEZ1_RIME', self.r_35mm, self.rho_r_max)
        self._accumulate_radius('R12_max_Rime', 'DRUEZ2_RIME', self.r_12mm, self.rho_r_max)
        self._accumulate_radius('R315_max_Rime', 'SAVADJIEV1_RIME', self.r_315kV, self.rho_r_max)


        # Accrétion cumulative de masse pour le verglas
        self.event_file['ACC_DRUEZ1'] = self.event_file['DRUEZ1'].cumsum()
        self.event_file['ACC_DRUEZ2'] = self.event_file['DRUEZ2'].cumsum()
        self.event_file['ACC_SAVADJIEV1'] = self.event_file['SAVADJIEV1'].cumsum()
        
        # Accrétion cumulative de masse pour le givre
        self.event_file['ACC_DRUEZ1_RIME'] = self.event_file['DRUEZ1_RIME'].cumsum()
        self.event_file['ACC_DRUEZ2_RIME'] = self.event_file['DRUEZ2_RIME'].cumsum()
        self.event_file['ACC_SAVADJIEV1_RIME'] = self.event_file['SAVADJIEV1_RIME'].cumsum()
        
        # === ALGORITHME 9-13-20-23 ===
        self._apply_hq_algo('TEMPEXT')
        self._accumulate_algo_hq('ACC_RAJ_FzRa','HQ_MASS',self.r_35mm)


        colonnes_rayon = [col for col in self.event_file.columns if col.endswith('_FzRa') or col.endswith('_Rime')]
        for col in colonnes_rayon:
            self.event_file[f'{col}_0mm'] = (self.event_file[col] - self.event_file[col].iloc[0]) * 1000
        
                
    #=================
    # ÉQUATIONS
    #=================
    def _accumulate_radius(self, col_name, taux_col, r0, rho):
        self.event_file[col_name] = 0.0
        self.event_file.at[self.event_file.index[0], col_name] = r0
        for i in range(1, len(self.event_file)):
            taux = self.event_file.iloc[i][taux_col]
            r_prev = self.event_file.iloc[i - 1][col_name]
            self.event_file.iat[i, self.event_file.columns.get_loc(col_name)] = self.current_radius(taux, r_prev, rho)

    # Équations verglas
    def eq_1(self, NA): return (0.0071 * NA) + 0.0254 if NA > 0 else 0 #Druez, d35 mm
    def eq_2(self, NA): return (0.00587 * NA) + 0.0205 if NA > 0 else 0 #Druez, d12.5 mm
    def eq_3(self, NA): return (0.0146 * NA) - 0.0124 if NA > 0 else 0 #Savadjiev, 315kV

    # Équations givre
    def eq_4(self, NA): return (0.00139 * NA) + 0.01899 if NA > 0 else 0 #Druez, d35 mm
    def eq_5(self, NA): return (0.00298 * NA) + 0.01997 if NA > 0 else 0 #Druez, d12.5 mm
    def eq_6(self, NA): return (0.0146 * NA) - 0.0124 if NA > 0 else 0 #Savadjiev, 315kV
    
    # Équation HQ / Rajaonarivelo
    def _apply_hq_algo(self, temp_column):
        temps=self.event_file[temp_column]
        na = self.event_file['NA']
        masses = []
        codes = []
        
        for i in range(0, len(na)):
            
            # Si il n'y a pas eu d'alarme dans la dernière heure, 
            # on utilise les codes standards (fonction first_code de hq_algo)
            if ( i == 0 ) or ((na.iloc[i-1] == 0) and (na.iloc[i] != 0) ):      
                # Initialisation avec first_code
                last_code = hq_algo.first_code(temps.iloc[i])
                codes.append(last_code)
                new_code, new_mass = hq_algo.raj(last_code, temps.iloc[i])
                masses.append(new_mass * na.iloc[i]) # kg/m*h
                
            elif (na.iloc[i-1] != 0) and (na.iloc[i] != 0):
                current_temp = temps.iloc[i]
                last_code = codes[-1]
                new_code, new_mass = hq_algo.raj(last_code, current_temp)
                 
                codes.append(new_code)
                masses.append(new_mass * na.iloc[i])
            
            else:
                new_code = 0
                new_mass = 0
                codes.append(new_code)
                masses.append(new_mass)
        
        # Ajout des colonnes au DataFrame
        self.event_file['HQ_CODE'] = codes
        self.event_file['HQ_MASS'] = masses
        
        
    
    def _accumulate_algo_hq(self, col_name, taux_col, r0):
        #TODO
        rho_from_code = {0:0,10:900,12:800,21:600,20:300}
        self.event_file[col_name] = 0.0
        self.event_file.at[self.event_file.index[0], col_name] = r0
        
        for i in range(1, len(self.event_file)):
            taux = self.event_file.iloc[i][taux_col]
            r_prev = self.event_file.iloc[i - 1][col_name]
            rho = rho_from_code[self.event_file.iloc[i]['HQ_CODE']]
            self.event_file.iat[i, self.event_file.columns.get_loc(col_name)] = self.current_radius(taux, r_prev, rho)

    def accretion(self, taux, r_prev, rho):
        return math.sqrt((taux / (rho * self.pi * self.wirel)) + r_prev**2)

    def current_radius(self, taux, r_prev, rho):
        return self.accretion(taux, r_prev, rho) if taux > 0 else r_prev
    
    #=================
    #GETTERS
    #=================
    def get_event_dataframe(self):
        return self.event_file
            


# Choix et exportation des dataframes selon des intervalles de temps définies

### TEST

In [4]:
stjovite_2022 = SygivreStation("STJOVITE_C",2022)

In [5]:
stjo_event1=stjovite_2022.set_event_dates((2022,3,1),(2022,3,15))

In [6]:
stjo_event1.head()


,VERGLAS,NA,TEMPEXT
timestamp,,,
2022-03-01 01:00:00,1620.0,0.0,-15.45
2022-03-01 02:00:00,1620.0,0.0,-14.84
2022-03-01 03:00:00,1620.0,0.0,-15.09
2022-03-01 04:00:00,1620.0,0.0,-15.15
2022-03-01 05:00:00,1620.0,0.0,-14.92


In [7]:
# verglas_event_stjovite = VerglasEvent(stjo_event1,"ST-JOVITE")


In [8]:
# df = verglas_event_stjovite.get_event_dataframe()

In [9]:
# df[df['HQ_CODE']>0][['TEMPEXT','NA','HQ_CODE','HQ_MASS','ACC_RAJ_FzRa','ACC_RAJ_FzRa_0mm','R35_FzRa_0mm','R35_min_Rime_0mm']]

## Analyse, évènement 1 (5-6 avril 2023)
#### Pour comparer à UQAM_PK, on utilise les stations CHENIER et HERTEL
#### Pour comparer à Morin Heights, on utilise les stations CHENIER, STJOVITE et LANAUD

*Section laissée en 'Markdown' pour éviter le coût d'exécution une fois les df produits*

In [10]:
save_file_output = "/chinook/larocque/DATA_HQ/EventFiles/"
event_name = "avril2023"


In [11]:
### CHENIER_C

In [12]:
### CHENIER_C
station = "CHENIER_C"

In [13]:
chenier_2023 = SygivreStation(station,2023).set_event_dates((2023,4,5),(2023,4,7))

In [14]:
chenier_2023.iloc[8:15]

,VERGLAS,NA,TEMPEXT
timestamp,,,
2023-04-05 09:00:00,631.0,0.0,-3.33
2023-04-05 10:00:00,631.0,0.0,-4.01
2023-04-05 11:00:00,633.0,2.0,-3.77
2023-04-05 12:00:00,633.0,0.0,-2.87
2023-04-05 13:00:00,633.0,0.0,-2.31
2023-04-05 14:00:00,633.0,0.0,-2.25
2023-04-05 15:00:00,635.0,2.0,-1.15


In [15]:
chenier_event = VerglasEvent(chenier_2023, station)

In [16]:
chenier_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-04-05 01:00:00,631.0,0.0,-0.28,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 02:00:00,631.0,0.0,-0.97,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 03:00:00,631.0,0.0,-1.57,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 04:00:00,631.0,0.0,-2.22,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 05:00:00,631.0,0.0,-2.46,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 06:00:00,631.0,0.0,-2.81,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 07:00:00,631.0,0.0,-3.08,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 08:00:00,631.0,0.0,-3.05,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 09:00:00,631.0,0.0,-3.33,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
# Saving CSV
filename = station+"_"+event_name+".csv"

chenier_event.get_event_dataframe().to_csv(save_file_output+filename)

In [18]:
### HERTEL_C

In [19]:
station = "HERTEL_C"

In [20]:
hertel_2023 = SygivreStation(station,2023).set_event_dates((2023,4,5),(2023,4,7))

In [21]:
#chenier_2023

In [22]:
hertel_event = VerglasEvent(hertel_2023, station)

In [23]:
hertel_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-04-05 01:00:00,304.0,0.0,0.50,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 02:00:00,304.0,0.0,-0.45,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 03:00:00,304.0,0.0,-0.87,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 04:00:00,304.0,0.0,-1.10,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 05:00:00,304.0,0.0,-1.19,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 06:00:00,304.0,0.0,-1.59,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 07:00:00,304.0,0.0,-1.59,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 08:00:00,304.0,0.0,-1.52,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 09:00:00,304.0,0.0,-0.88,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
# Saving CSV
filename = station+"_"+event_name+".csv"

hertel_event.get_event_dataframe().to_csv(save_file_output+filename)

In [25]:
### STJOVITE_C

In [26]:
station = "STJOVITE_C"

In [27]:
stjovite_2023 = SygivreStation(station,2023).set_event_dates((2023,4,5),(2023,4,7))

In [28]:
#chenier_2023

In [29]:
stjovite_event = VerglasEvent(stjovite_2023, station)

In [30]:
stjovite_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-04-05 01:00:00,1800.0,0.0,-3.94,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 02:00:00,1800.0,0.0,-4.24,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 03:00:00,1800.0,0.0,-4.46,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 04:00:00,1800.0,0.0,-5.07,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 05:00:00,1800.0,0.0,-5.26,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 06:00:00,1800.0,0.0,-5.34,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 07:00:00,1800.0,0.0,-5.54,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 08:00:00,1800.0,0.0,-5.62,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 09:00:00,1802.0,2.0,-6.33,0.0396,0.03224,0.0168,0.017896,0.007104,0.057552,0.02177,...,0.395688,0.853877,0.051644,0.647966,1.909356,0.154795,0.245760,0.777070,0.058097,0.389758


In [31]:
# Saving CSV
filename = station+"_"+event_name+".csv"

stjovite_event.get_event_dataframe().to_csv(save_file_output+filename)

In [32]:
### LANAUD_C

In [33]:
station = "LANAUD_C"

In [34]:
lanaudiere_2023 = SygivreStation(station,2023).set_event_dates((2023,4,5),(2023,4,7))

In [35]:
#chenier_2023

In [36]:
lanaudiere_event = VerglasEvent(lanaudiere_2023, station)

In [37]:
lanaudiere_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-04-05 01:00:00,484.0,0.0,-1.46,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 02:00:00,484.0,0.0,-2.04,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 03:00:00,484.0,0.0,-2.33,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 04:00:00,484.0,0.0,-2.70,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 05:00:00,484.0,0.0,-3.13,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 06:00:00,484.0,0.0,-3.44,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 07:00:00,484.0,0.0,-3.70,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 08:00:00,484.0,0.0,-3.97,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-04-05 09:00:00,484.0,0.0,-4.35,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [38]:
# Saving CSV
filename = station+"_"+event_name+".csv"

lanaudiere_event.get_event_dataframe().to_csv(save_file_output+filename)

## Analyse, évènement 2 (6 mars 2022)
#### Pour comparer à UQAM_PK, on utilise les stations CHENIER et HERTEL
#### Pour comparer à Trois-Rivières, on utilise les stations GRANDPRE et MAURICIE

In [39]:
save_file_output = "/chinook/larocque/DATA_HQ/EventFiles/"
event_name = "mars2022"
start = (2022,3,5)
end = (2022,3,7)

In [40]:
### MONTRÉAL

In [41]:
### CHENIER_C

In [42]:
### CHENIER_C
station = "CHENIER_C"

In [43]:
chenier_2022 = SygivreStation(station,2022).set_event_dates(start,end)

In [44]:
chenier_event = VerglasEvent(chenier_2022, station)

In [45]:
chenier_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-05 01:00:00,574.0,0.0,-9.21,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 02:00:00,574.0,0.0,-8.30,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 03:00:00,574.0,0.0,-8.61,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 04:00:00,574.0,0.0,-10.18,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 05:00:00,574.0,0.0,-9.33,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 06:00:00,574.0,0.0,-10.49,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 07:00:00,574.0,0.0,-12.38,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 08:00:00,574.0,0.0,-15.70,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 09:00:00,574.0,0.0,-12.19,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
# Saving CSV
filename = station+"_"+event_name+".csv"

chenier_event.get_event_dataframe().to_csv(save_file_output+filename)

In [47]:
### HERTEL_C

In [48]:
station = "HERTEL_C"

In [49]:
hertel_2022 = SygivreStation(station,2022).set_event_dates(start,end)

In [50]:
hertel_event = VerglasEvent(hertel_2022, station)

In [51]:
hertel_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-05 01:00:00,245.0,0.0,-6.85,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 02:00:00,245.0,0.0,-6.61,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 03:00:00,245.0,0.0,-7.30,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 04:00:00,245.0,0.0,-8.24,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 05:00:00,245.0,0.0,-8.53,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 06:00:00,245.0,0.0,-8.74,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 07:00:00,245.0,0.0,-9.72,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 08:00:00,245.0,0.0,-9.55,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-05 09:00:00,245.0,0.0,-8.18,0,0,0,0.0175,0.00625,0.0575,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
# Saving CSV
filename = station+"_"+event_name+".csv"

hertel_event.get_event_dataframe().to_csv(save_file_output+filename)

In [53]:
### TROIS-RIVIÈRES

In [54]:
### GRANDPRE

In [55]:
station = "GRANDPRE_C"

In [56]:
grandpre_2022 = SygivreStation(station,2022).set_event_dates(start,end)

In [57]:
grandpre_event = VerglasEvent(grandpre_2022, station)

In [58]:
grandpre_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-05 01:00:00,244.0,0.0,-12.81,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 02:00:00,244.0,0.0,-13.93,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 03:00:00,244.0,0.0,-14.87,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 04:00:00,244.0,0.0,-15.28,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 05:00:00,244.0,0.0,-15.39,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 06:00:00,244.0,0.0,-16.97,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 07:00:00,244.0,0.0,-15.91,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 08:00:00,244.0,0.0,-16.17,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 09:00:00,244.0,0.0,-14.02,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [59]:
# Saving CSV
filename = station+"_"+event_name+".csv"

grandpre_event.get_event_dataframe().to_csv(save_file_output+filename)

In [60]:
### MAURICIE_C

In [61]:
station = "MAURICIE_C"

In [62]:
mauricie_2022 = SygivreStation(station,2022).set_event_dates(start,end)

In [63]:
mauricie_event = VerglasEvent(mauricie_2022, station)

In [64]:
mauricie_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-03-05 01:00:00,844.0,0.0,-9.45,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 02:00:00,844.0,0.0,-9.34,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 03:00:00,844.0,0.0,-9.67,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 04:00:00,844.0,0.0,-9.96,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 05:00:00,844.0,0.0,-11.02,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 06:00:00,844.0,0.0,-10.91,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 07:00:00,844.0,0.0,-11.86,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 08:00:00,844.0,0.0,-11.87,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-03-05 09:00:00,844.0,0.0,-11.15,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [65]:
# Saving CSV
filename = station+"_"+event_name+".csv"

mauricie_event.get_event_dataframe().to_csv(save_file_output+filename)

## Analyse, évènement 3 (24 janvier 2024)
#### Pour comparer à UQAM_PK, on utilise les stations CHENIER et HERTEL


In [66]:
save_file_output = "/chinook/larocque/DATA_HQ/EventFiles/"
event_name = "janvier2024"
start = (2024,1,24)
end = (2024,1,26)

### CHENIER_C

In [67]:
### CHENIER_C
station = "CHENIER_C"

In [68]:
chenier_2024 = SygivreStation(station,2024).set_event_dates(start,end)

In [69]:
chenier_event = VerglasEvent(chenier_2024, station)

In [70]:
chenier_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-24 01:00:00,653.0,0.0,-10.56,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 02:00:00,653.0,0.0,-10.99,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 03:00:00,653.0,0.0,-11.02,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 04:00:00,653.0,0.0,-11.42,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 05:00:00,653.0,0.0,-11.35,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 06:00:00,653.0,0.0,-11.14,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 07:00:00,653.0,0.0,-11.25,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 08:00:00,653.0,0.0,-11.38,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 09:00:00,653.0,0.0,-11.07,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [71]:
# Saving CSV
filename = station+"_"+event_name+".csv"

chenier_event.get_event_dataframe().to_csv(save_file_output+filename)

### HERTEL_C

In [72]:
station = "HERTEL_C"

In [73]:
hertel_2024 = SygivreStation(station,2024).set_event_dates(start,end)

In [74]:
hertel_event = VerglasEvent(hertel_2024, station)

In [75]:
hertel_event.get_event_dataframe()

,VERGLAS,NA,TEMPEXT,DRUEZ1,DRUEZ2,SAVADJIEV1,R35_FzRa,R12_FzRa,R315_FzRa,DRUEZ1_RIME,...,R35_FzRa_0mm,R12_FzRa_0mm,R315_FzRa_0mm,R35_min_Rime_0mm,R12_min_Rime_0mm,R315_min_Rime_0mm,R35_max_Rime_0mm,R12_max_Rime_0mm,R315_max_Rime_0mm,ACC_RAJ_FzRa_0mm
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-24 01:00:00,351.0,0.0,-9.92,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 02:00:00,351.0,0.0,-9.88,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 03:00:00,351.0,0.0,-9.98,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 04:00:00,351.0,0.0,-10.50,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 05:00:00,351.0,0.0,-10.76,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 06:00:00,351.0,0.0,-10.86,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 07:00:00,351.0,0.0,-10.68,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 08:00:00,351.0,0.0,-10.41,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2024-01-24 09:00:00,351.0,0.0,-9.96,0.0000,0.00000,0.0000,0.017500,0.006250,0.057500,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [76]:
# Saving CSV
filename = station+"_"+event_name+".csv"

hertel_event.get_event_dataframe().to_csv(save_file_output+filename)